<a href="https://colab.research.google.com/github/tonyjung01/Machine-Learning/blob/main/5-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

## 검증 세트

depth를 조절해가며 성능 괜찮은 걸 찾으려고 했는데 이러면 테스트 세트에 맞는 모델이 만들어지는 문제 발생  
검증 세트를 사용하자

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

여기까지는 원래 하던 거

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

한번 더 train_test_split을 써서 train_input의 20%를 val_input으로 만듬

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


검증 세트 써보니 확실히 훈련 세트에 과대적합되어 있음

## 교차 검증

검증 세트를 만들면 훈련 세트가 줄어드는데, 훈련 세트가 많아야 좋은 모델이 만들어짐  
그렇다고 검증 세트 너무 적으면 검증 점수 불안정해짐  
이럴 때 교차 검증 사용

k 폴드 교차 검증 : 데이터 k개로 나누고 1개씩 돌아가면서 테스트 세트로 활용하여 점수 계산  
k번 평가한 점수의 평균을 실제 점수로 사용

In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00968242, 0.0090158 , 0.00888729, 0.01025009, 0.00867534]), 'score_time': array([0.00165534, 0.00157309, 0.00139976, 0.00160861, 0.0013938 ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


cross_validate 디폴트 5폴드

In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


그런데 이슈는 cross_validate는 따로 섞지 않아서 spliter로 섞어줘야됨

In [9]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝

매개변수는 서로 연관성이 있어서 하나가 최적인 걸 찾아도 다른 걸 바꾸면 최적이 아니게 되어버림  
이를 위해서 그리드 서치 사용  
그리드 서치는 하이퍼파라미터 탐색과 교차검증 함께 수행함

In [11]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

최적의 Min_impurity_decrease를 찾기 위해 5개의 값 X 5fold 총 25번 모델 훈련  
n_jobs에서 병렬 실행에 사용할 CPU 코어 지정 가능  
기본이 1, -1로 하면 모든 코어 사용

In [15]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


그리드 서치는 훈련이 끝나면 교차 검증 점수가 가장 높은 모델의 매개변수 조합으로 자동으로 모델 다시 훈련함  
이는 gs 객체의 best_estimator에 저장

In [16]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


최고의 매개변수 값은 best_params_에 저장

In [17]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


각 매겨변수로 시행한 교차 검증의 평균점수  
argmax로 가장 큰 거 뽑아도 됨

In [18]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


과정을 정리해보자면,

1.   탐색할 매개변수 지정
2.   훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합 찾기(그리드 서치 객체에 자동 저장)
3.   그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델 훈련(그리드 서치 객체에 저장)



In [21]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

교차 검증 횟수는 9 X 15 X 10 = 1350번  
5 폴드 교차 검증이므로 모델의 수는 1350 X 5 = 6750

In [23]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [24]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [25]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


그리드 서치로 일일이 매개변수 바꿔갈 필요 없이 자동으로 교차 검증 수행해서 최상의 매개변수 찾기 가능  
그런데 매개변수 간격은 어케 정하지?

### 랜덤 서치

랜덤 서치는 매개변수 값의 목록을 전달하지 않고 매개변수 샘플링할 확률분포 객체 전달

In [26]:
from scipy.stats import uniform, randint

randint는 정수, uniform은 실수값을 균등 분포에서 샘플링함

In [27]:
rgen = randint(0, 10)
rgen.rvs(10)

array([8, 8, 0, 0, 6, 9, 3, 0, 0, 4])

In [28]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 84, 100,  94, 109,  85, 100, 119, 100, 113,  96]))

1000개를 샘플링해서 숫자 세어 보면 얼추 고르게 추출됨

In [29]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.22061098, 0.59965214, 0.98966083, 0.19489325, 0.5651825 ,
       0.83488513, 0.24305798, 0.88682497, 0.69581132, 0.86794267])

In [30]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [31]:
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab755429c90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ab755429fd0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab755342f10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab75542ad50>},
                   random_state=42)

이전보다 훨씬 줄어든 100회 샘플링 교차 검증 수행

In [32]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [33]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


교차 검증 점수 확인

In [34]:
dt = gs.best_estimator_

print(dt.score(test_input, test_target))

0.86


## 확인문제

In [35]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random', random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab755429c90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ab755429fd0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab755342f10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ab75542ad50>},
                   random_state=42)

splitter 매개변수의 디폴트는 best인데 random으로 해봄

In [36]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 43, 'min_impurity_decrease': 0.00011407982271508446, 'min_samples_leaf': 19, 'min_samples_split': 18}
0.8458726956392981
0.786923076923077


성능 감소